In [7]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv(r"C:\Users\Vai.Mathi\OneDrive\Documents\Neural Network from Scratch\digit-recognizer\train.csv")

In [8]:
#We will have m examples each with 28 by 28 (784) pixels
data = np.array(data)
m, n = data.shape
# shuffle before splitting into dev and training sets
np.random.shuffle(data) 


#Tranpose the matrix so that each column is an example
#Splits set into dev and train set. Y_dev is the number it represents and X_dev is the binary value for each pixel
#Neural Network will predict based on following structure: 
#0th (the input) layer will consist of 784 nodes, 1st layer (hidden-layer) , 2nd layer (output layer) will be used to measure the probability that a given node is a particular number
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [9]:
Y_train

array([8, 0, 9, ..., 1, 0, 0], dtype=int64)

In [10]:
#W1 is a 10 by 784 matrix corresponding to 10 neurons to the first hidden layer
#A_0 = 784 x M (where m is the number of images)
#Z_1 = w_1 * A_0 + B_1 . Z_1 is a 10 x M matrix  where 10 is the number of neurons
#B_1 is  a 10 x 1 matrix
#A_1 will equal the output of the activation function with Z_1 as its input. 
#For this project I will use the  rectified linear unit (ReLU) activation function because its computationally cheap and addresses the vanishing gradient problem
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

#Computes probabilities of each neuron using their scores to determine their proportion against other neurons
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

#Equations:
#dZ_L = A_L - Y
    #A_L = Predicted output
    #Y = True-value
    #dZ_L = loss with respect to the final layer's linear combination of weights and inputs. It is the rror
        #A low dZ_L means we don't adjust the weights that much

    #Intuition: This is how  incorrect our prediciton is
#dW_L =(dZ_L * A_L-1.T) / m
    #dZ_L = gradient
    #A_L-1 = Activation from previous layer
        #we transpose to allow  for proper matrix multiplication
    #m = number of images

    #dW_L = the change in weights

    #Intuition: The change in weights should equal the activations of the previous layer multiplied by the gradient (the incorrectness) over the number of images we have
#db_L = sigma(dZ_L) / m
    #dZ_L = gradient
    #m = number of images

    #db_L = change in bias

    #Intuition: the gradient of the bias is the average of all the gradients for each example

#dA_L-1 = W_L.T * dZ_L
    #dA_L-1 = the gradient loss with respect to the activations from the last layer
    #W_L.T = tranpose of the weight matrix in the current layer
    #dZ_L = the gradient loss

    #Intuition: the errror from the current layer is sent backways to the previous layer and weighted by strength of connections

#dZ_L-1 = dA_L-1 (dot) g'(Z_L-1)
    #dA_L-1 = the gradient loss with respect to the activations from the last layer
    #g'(Z_L-1) = the derivative of the activation function (ReLU) with the pre-activation ivalue Z_L-1 as an input

    #Intuition: The gradient with respect to the pre-activation of the previous layer is the dot product of the incming gradient 
    # and the derivative of the activation function. This adjusts the error signal based on how sensitive the activation function is.

#dW_L-1 = (1/m) * dZ_L-1 * A_l-2.T
    #dZ_L-1 = rate of change of the pre-activation values
    #A_L-2 = activations from layer before
    #1/m averages these gradients 




#ReLU is either a constant value (0) or its a linear-equation so ReLU deriv is  either 0 or the slope of the linear equation which is 1
def ReLU_deriv(Z):
    return Z > 0

#Prepares y-value for backprop
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
#See Equation explanation
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

#Adjusts weights and betas based on arbitrary alpha (learning rate)
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [13]:
# Return the index of the highest score, which is the predicted class
def get_predictions(A2):
    return np.argmax(A2, axis=0)  


def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            
            predictions = get_predictions(A2)
            accuracy, precision, recall, f1_score = get_accuracy_metrics(Y, predictions)

            print("Iteration: ", i, ', Accuracy:', round(accuracy, 4))
            

        
    return W1, b1, W2, b2

In [14]:

W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 501)


Iteration:  0 , Accuracy: 0.1344
Iteration:  50 , Accuracy: 0.5405
Iteration:  100 , Accuracy: 0.6859
Iteration:  150 , Accuracy: 0.7528
Iteration:  200 , Accuracy: 0.7855
Iteration:  250 , Accuracy: 0.8057
Iteration:  300 , Accuracy: 0.8224
Iteration:  350 , Accuracy: 0.8336
Iteration:  400 , Accuracy: 0.8436
Iteration:  450 , Accuracy: 0.8505
Iteration:  500 , Accuracy: 0.8559
